In [8]:
pip install langchain langchain-openai pandas


   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   -------------- ------------------------- 4.2/11.6 MB 27.9 MB/s eta 0:00:01
   ----------------------------- ---------- 8.7/11.6 MB 25.6 MB/s eta 0:00:01
   ------------------------------------ --- 10.7/11.6 MB 19.2 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 15.5 MB/s eta 0:00:00


In [27]:
import pandas as pd

from main import create_prompt_user_response_matrix, get_openai_response, user_response_dict
from quality_of_life_pydantic import QualityOfLifeIndexes

In [28]:
prompt = create_prompt_user_response_matrix(user_response_dict)
response = get_openai_response(prompt)
response

QualityOfLifeIndexes(a_sense_of_control=SenseOfControl(cost_of_living=70, essential_services=80, safety=85, influence_and_contribution=60), a_sense_of_wonder=SenseOfWonder(distinctive_design_and_culture=65, play_and_recreation=75), connected_communities=ConnectedCommunities(belonging=75, local_business_and_jobs=65), connection_to_nature=ConnectionToNature(biodiversity=70, climate_resilience_and_adaptation=65, green_and_blue_spaces=75), getting_around=GettingAround(walking_and_cycling=55, public_transport=70, car=75), health_equity=HealthEquity(housing_standard=70, air_noise_light=75, food_choice=65))

In [59]:
def read_csv(path='data/irl_ed_qol.csv'):
    df = pd.read_csv(path)
    return df

df = read_csv()
df

,GUID,Electoral Divisions,Population (2022) - F1060C01,ED_ID,ED,COUNTY,PROVINCE,CENTROID_X,CENTROID_Y,AREA,...,connected_communities:local_business_and_jobs,connection_to_nature:biodiversity,connection_to_nature:climate_resilience_and_adaptation,connection_to_nature:green_and_blue_spaces,getting_around:car,getting_around:public_transport,getting_around:walking_and_cycling,health_equity:air_noise_light,health_equity:food_choice,health_equity:housing_standard
0,2ae19629-1836-13a3-e055-000000000001,"Agha, Carlow",419,17001,AGHA,CARLOW,Leinster,673756.71,665678.08,12316519.48,...,54,71,65,79,58,47,61,69,64,48
1,2ae19629-1837-13a3-e055-000000000001,"Ballinacarrig, Carlow",1122,17002,BALLINACARRIG,CARLOW,Leinster,672629.70,672908.20,16250711.99,...,52,69,63,77,56,45,59,67,62,46
2,2ae19629-1840-13a3-e055-000000000001,"Ballintemple, Carlow",599,17003,BALLINTEMPLE,CARLOW,Leinster,688760.24,666672.30,28342590.20,...,63,80,74,88,67,56,70,78,73,57
3,2ae19629-1841-13a3-e055-000000000001,"Ballon, Carlow",788,17004,BALLON,CARLOW,Leinster,683124.29,665181.68,9789628.09,...,50,67,61,75,54,43,57,65,60,44
4,2ae19629-1842-13a3-e055-000000000001,"Ballyellin, Carlow",451,17005,BALLYELLIN,CARLOW,Leinster,671711.49,653811.90,15793618.59,...,56,74,68,82,61,49,64,72,67,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,2ae19769-1f5b-13a3-e055-500000000001,"Matehy, Cork City",1772,4725019,MATEHY,CORK,Munster,557571.58,575504.26,5101820.95,...,72,68,65,76,64,71,78,67,80,61
3388,2ae19819-1f5b-13a3-e055-110000000001,"Ovens, Cork City",1785,4727019,OVENS,CORK,Munster,556639.96,570280.17,673434.60,...,75,71,68,79,67,74,81,70,83,64
3389,2ae19779-1f5b-13a3-e055-600000000001,"Rathcooney, Cork City",8707,4727719,RATHCOONEY,CORK,Munster,569762.55,575727.11,21649141.73,...,71,67,64,75,63,70,77,66,79,60
3390,2ae19629-1f5b-13a3-e055-100000000001,"Riverstown, Cork City",3946,4727919,RIVERSTOWN,CORK,Munster,572444.39,577456.63,5281507.55,...,60,60,65,55,60,70,65,60,70,50


In [61]:
def get_primary_score(row, user_qol_sorted):
    score = 0
    for qol_factor, qol_score in user_qol_sorted:
        score = score + (abs(qol_score - row[qol_factor]))
    return score
    
def get_secondary_score(row, user_qol_sorted):
    score = 0
    for qol_factor, qol_score in user_qol_sorted:
        score = score + (abs(qol_score - row[qol_factor]))
    return score

In [64]:
def get_top_locations(df_qol: pd.DataFrame, user_qol: QualityOfLifeIndexes) -> pd.DataFrame :
    """
    Function to get top locations based on a QualityOfLifeIndex dataframe and user responses
    :param df_qol: dataframe containing qol matrix for all regions
    :param user_qol: QualityOfLifeIndexes object containing user responses
    :return: dataframe containing qol matrix for top 5 matching regions
    """
    user_qol_dict = pd.json_normalize(user_qol.dict(), sep=":").to_dict(orient='records')[0]
    user_qol_sorted = sorted(user_qol_dict.items(), key=lambda x: x[1], reverse=True)
    
    df_qol['user_score_primary'] = df_qol.apply(lambda x: get_primary_score(x, user_qol_sorted[0:5]), axis=1)
    df_qol['user_score_secondary'] = df_qol.apply(lambda x: get_secondary_score(x, user_qol_sorted), axis=1)
    df_qol = df_qol.sort_values(by=['user_score_primary', 'user_score_secondary', 'QoL'])
    return df_qol.iloc[0:5]

df_qol = get_top_locations(df, response)
df_qol

,GUID,Electoral Divisions,Population (2022) - F1060C01,ED_ID,ED,COUNTY,PROVINCE,CENTROID_X,CENTROID_Y,AREA,...,connection_to_nature:climate_resilience_and_adaptation,connection_to_nature:green_and_blue_spaces,getting_around:car,getting_around:public_transport,getting_around:walking_and_cycling,health_equity:air_noise_light,health_equity:food_choice,health_equity:housing_standard,user_score_primary,user_score_secondary
3189,2ae19629-1cb0-13a3-e055-000000000001,"Swords-Lissenhall, Fingal",12065,267133,SWORDS-LISSENHALL,DUBLIN,Leinster,717020.91,750005.01,18775562.41,...,71,73,69,78,82,65,79,61,7,122
3084,2ae19629-1cc6-13a3-e055-000000000001,"Blanchardstown-Blakestown, Fingal",43905,267028,BLANCHARDSTOWN-BLAKESTOWN,DUBLIN,Leinster,704404.81,739450.32,7873486.98,...,66,72,65,76,78,70,76,58,7,122
3164,2ae19629-1de3-13a3-e055-000000000001,"Palmerston Village, South Dublin",3689,267108,PALMERSTON VILLAGE,DUBLIN,Leinster,708538.55,734472.19,1168197.49,...,68,81,67,63,79,72,76,62,8,105
3169,2ae19629-1e00-13a3-e055-000000000001,"Rathfarnham-Ballyroan, South Dublin",2743,267113,RATHFARNHAM-BALLYROAN,DUBLIN,Leinster,713045.79,728301.45,685888.62,...,67,80,66,62,78,71,75,60,9,105
3083,2ae19629-1cc5-13a3-e055-000000000001,"Blanchardstown-Abbotstown, Fingal",6551,267027,BLANCHARDSTOWN-ABBOTSTOWN,DUBLIN,Leinster,709702.84,739126.91,7186209.01,...,68,74,67,78,80,72,78,60,9,128


In [66]:
user_qol_dict = pd.json_normalize(response.dict(), sep=":").to_dict(orient='records')[0]
user_qol_sorted = sorted(user_qol_dict.items(), key=lambda x: x[1], reverse=True)
len(user_qol_sorted)

17

In [46]:
df_user_qol = pd.DataFrame(response.dict())
df_user_qol_dict = pd.json_normalize(response.dict(), sep=":").to_dict(orient='records')[0]
l = sorted(df_user_qol_dict.items(), key=lambda x: x[1], reverse=True)
s = 0
for x in l:
    s = s + x[1]
s


1195

In [23]:

df_user_qol_dict

{'a_sense_of_control:cost_of_living': {0: 75},
 'a_sense_of_control:essential_services': {0: 80},
 'a_sense_of_control:safety': {0: 85},
 'a_sense_of_control:influence_and_contribution': {0: 60},
 'a_sense_of_wonder:distinctive_design_and_culture': {0: 65},
 'a_sense_of_wonder:play_and_recreation': {0: 70},
 'connected_communities:belonging': {0: 75},
 'connected_communities:local_business_and_jobs': {0: 65},
 'connection_to_nature:biodiversity': {0: 70},
 'connection_to_nature:climate_resilience_and_adaptation': {0: 65},
 'connection_to_nature:green_and_blue_spaces': {0: 75},
 'getting_around:walking_and_cycling': {0: 60},
 'getting_around:public_transport': {0: 70},
 'getting_around:car': {0: 80},
 'health_equity:housing_standard': {0: 75},
 'health_equity:air_noise_light': {0: 70},
 'health_equity:food_choice': {0: 65}}

In [ ]:
{
    "a_sense_of_control": {
        "cost_of_living": 65,
        "safety": 78,
        "influence_and_contribution": 55
    },
    "health_equity": {
        "housing_standard": 58,
        "air_noise_light": 82,
        "food_choice": 62
    },
    "connection_to_nature": {
        "green_and_blue_spaces": 75,
        "biodiversity": 68,
        "climate_resilience_and_adaptation": 72
    },
    "a_sense_of_wonder": {
        "distinctive_design_and_culture": 60,
        "play_and_recreation": 70
    },
    "getting_around": {
        "walking_and_cycling": 52,
        "public_transport": 48,
        "car": 68
    },
    "connected_communities": {
        "belonging": 72,
        "local_business_and_jobs": 58
    },
}

In [67]:
## Clean up dataset
df = read_csv('data/irl_ed_qol.csv')
for qol_factor, qol_score in user_qol_sorted:
    df[qol_factor] = df[qol_factor].clip(0,100)
df.to_csv('data/irl_ed_qol.csv', index=False)